In [1]:
import numpy as np
import pandas as pd
import statistics
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_pacf
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA 
from sklearn.metrics import mean_squared_error

In [2]:
#read file
df_data = pd.read_csv("data/dataset_mood_smartphone.csv")
df_data.head()

,Unnamed: 0,id,time,variable,value
0,1,AS14.01,2014-02-26 13:00:00.000,mood,6.0
1,2,AS14.01,2014-02-26 15:00:00.000,mood,6.0
2,3,AS14.01,2014-02-26 18:00:00.000,mood,6.0
3,4,AS14.01,2014-02-26 21:00:00.000,mood,7.0
4,5,AS14.01,2014-02-27 09:00:00.000,mood,6.0


In [3]:
prev_day="2014-02-26"
new_data=pd.DataFrame(columns=['id', 'time', 'var',"value"])
temp_list=[]
for index, row in df_data.iterrows():
    date=row["time"].split(" ")[0]
    if date != prev_day:
        new_data.loc[index]=[df_data.loc[index-1]["id"], prev_day , df_data.loc[index-1]["variable"], statistics.mean(temp_list)]
        temp_list=[row["value"]]
    else:
        temp_list.append(row["value"])
    prev_day=date

In [19]:
len(np.unique(np.array(df_data["id"])))

27

In [6]:
df_mood_id = new_data.loc[(new_data['var'] == "mood")]
df_time_value = df_mood_id[['id','time',"value"]].reset_index()
df_time_value['time'] = pd.to_datetime(df_time_value['time'],format="%Y-%m-%d")

X = []

id = 'AS14.01'
prev_index = 0
for i, row in df_time_value.iterrows():
    delta = (df_time_value['time'].iloc[prev_index] - df_time_value['time'].iloc[i]).days
    if delta < -1:
        for n in range(abs(delta)-1):
            X.append(None)
        X.append(df_time_value['value'].iloc[i])
    elif delta > 0:
        for n in range(365-delta-1):
            X.append(None)
        X.append(df_time_value['value'].iloc[i])
    else:
        X.append(df_time_value['value'].iloc[i])
    prev_index = i
            
#Set up the input for ARIMA
# index = df_time_value[["time"]].values
# data_np = df_time_value["value"].values
series = pd.Series(X)
X= series.values
print(X)
np.savetxt("dataset_arima.txt", X)


[6.25       6.33333333        nan ... 7.         6.8        7.        ]


In [7]:
X = [x for x in X if str(x) != 'nan']
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]

history = [x for x in train]
predictions = list()
for t in range(len(test)):
    #order=(p,d,q) 
    model = ARIMA(history, order=(1,1,1))#tunne parameters!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))
error = mean_squared_error(test, predictions)
print('Test MSE: %.3f' % error)

predicted=7.327354, expected=7.000000
predicted=7.139577, expected=6.800000
predicted=7.040528, expected=6.000000
predicted=6.692318, expected=6.333333
predicted=6.706455, expected=7.400000
predicted=7.062173, expected=7.000000
predicted=6.962265, expected=6.833333
predicted=6.904209, expected=7.500000
predicted=7.151157, expected=6.000000
predicted=6.631105, expected=7.000000
predicted=6.911271, expected=7.000000
predicted=6.926274, expected=7.400000
predicted=7.092064, expected=7.000000
predicted=6.989369, expected=7.250000
predicted=7.087075, expected=7.000000
predicted=7.018392, expected=7.500000
predicted=7.206924, expected=5.200000
predicted=6.381227, expected=6.400000
predicted=6.645581, expected=7.000000
predicted=6.834572, expected=7.600000
predicted=7.091578, expected=7.200000
predicted=7.020361, expected=7.400000
predicted=7.125310, expected=7.000000
predicted=7.016782, expected=6.200000
predicted=6.709586, expected=6.600000
predicted=6.782520, expected=6.800000
predicted=6.

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\base\model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


predicted=6.992812, expected=7.400000
predicted=7.046169, expected=7.200000
predicted=7.015679, expected=7.400000
predicted=7.115241, expected=7.600000
predicted=7.227788, expected=7.000000
predicted=7.047329, expected=6.600000
predicted=6.889438, expected=6.200000
predicted=6.700541, expected=7.600000
predicted=7.169819, expected=8.000000
predicted=7.377038, expected=7.200000
predicted=7.150796, expected=5.800000
predicted=6.623746, expected=5.400000
predicted=6.364812, expected=7.400000
predicted=7.015261, expected=6.400000
predicted=6.680808, expected=7.400000
predicted=7.026347, expected=7.000000
predicted=6.920314, expected=7.400000
predicted=7.081868, expected=7.000000
predicted=6.969027, expected=7.600000
predicted=7.200381, expected=7.600000
predicted=7.249425, expected=7.000000
predicted=7.064451, expected=7.400000
predicted=7.209030, expected=5.800000
predicted=6.625924, expected=7.000000
predicted=6.981154, expected=8.750000
predicted=7.643865, expected=7.600000
predicted=7.

predicted=7.115117, expected=7.000000
predicted=7.052712, expected=6.750000
predicted=6.955278, expected=7.000000
predicted=7.021943, expected=7.000000
predicted=7.019568, expected=7.333333
predicted=7.139281, expected=7.000000
predicted=7.041419, expected=7.000000
predicted=7.036695, expected=7.000000
predicted=7.032541, expected=7.000000
predicted=7.028888, expected=7.000000
predicted=7.025676, expected=8.000000
predicted=7.388062, expected=7.000000
predicted=7.097795, expected=8.000000
predicted=7.450578, expected=8.000000
predicted=7.519163, expected=8.400000
predicted=7.727158, expected=7.600000
predicted=7.517745, expected=7.000000
predicted=7.306446, expected=7.800000
predicted=7.563702, expected=7.333333
predicted=7.422529, expected=8.250000
predicted=7.748069, expected=7.666667
predicted=7.598504, expected=8.600000
predicted=7.950380, expected=8.000000
predicted=7.815176, expected=7.500000
predicted=7.655726, expected=8.000000
predicted=7.820219, expected=8.500000
predicted=8.

In [9]:
df_time_value.shape

(1268, 4)